# Main imports and code

In [ ]:
# check which gpu we're using
!nvidia-smi

Sun Feb 27 12:42:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip  install transformers
!pip install pytorch-ignite

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 33.6 MB/s 
     |████████████████████████████████| 895 kB 23.7 MB/s 
     |████████████████████████████████| 596 kB 12.9 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 251 kB 6.4 MB/s 


In [ ]:
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
from transformers import AutoTokenizer, AutoModel, AdamW


In [ ]:
!pip install sentencepiece


     |████████████████████████████████| 1.2 MB 6.6 MB/s 


In [ ]:
import random
import os
from urllib import request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test=pd.read_csv('/content/drive/MyDrive/ISarcasm/TestSet/task_B_En_test.csv')


# RoBERTa Baseline for Task 1

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
!pip install datasets


     |████████████████████████████████| 311 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 29.6 MB/s 
     |████████████████████████████████| 134 kB 32.6 MB/s 
     |████████████████████████████████| 212 kB 34.5 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 271 kB 38.1 MB/s 
     |████████████████████████████████| 144 kB 31.3 MB/s 


In [ ]:
class PCLTrainDataset(Dataset):
    def __init__(self, df, tokenizer, max_length,displacemnt):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        self.label=df[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # summary = self.summary[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
                            
        target = self.label[index]
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
        
       
        
        
        return {
            
            'text_ids': torch.tensor(text_ids, dtype=torch.long),
            'text_mask': torch.tensor(text_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.float)
        }


In [ ]:
class PCL_Model_Arch(nn.Module):
    def __init__(self,pre_trained='roberta-base'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained, output_hidden_states=True)
        # output_channel = 16  # number of kernels
        num_classes = 6  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        self.layer_norm = nn.LayerNorm(368640)
        self.dropout = nn.Dropout(0.5)
        self.dense = nn.Sequential(
            nn.Linear(368640, 256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

       
        self.softmax = nn.Sigmoid()

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.bert(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        hidden_layers = outputs[2]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        torch.cuda.empty_cache()
        x = hidden_layers[:, -7:-3] 
        torch.cuda.empty_cache()
        x = x.view(x.shape[0],-1)
        torch.cuda.empty_cache()
        x = self.layer_norm(x)
        torch.cuda.empty_cache()
        x = self.dropout(x)
        torch.cuda.empty_cache()
        x = self.dense(x)
      
        torch.cuda.empty_cache()
        # x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        # x = [F.dropout(i) for i in x]
        # # max-over-time pooling; # (batch, channel_output) * ks
        # x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # # concat results; (batch, channel_output * ks)
        # x = torch.cat(x, 1)
        # add dropout
       
        return x

In [ ]:
!pip install emoji

     |████████████████████████████████| 174 kB 5.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=754e89add53c04b6950f90cffce24cd2d49e3f9646d3bab6ca3b29b342328d92
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
tokenizer= AutoTokenizer.from_pretrained('roberta-base')


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
CONFIG = {"seed": 2021,
          "epochs": 5,
          "model_name": "xlnet-base-cased",
          "train_batch_size": 16,
          "valid_batch_size": 16,
          "max_length": 120,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [ ]:
test.dropna(inplace=True)

In [ ]:
valid_dataset = PCLTrainDataset(test, tokenizer=tokenizer, max_length=CONFIG['max_length'],displacemnt=0)
valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['text_ids'].to(device, dtype = torch.long)
        mask = data['text_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        sig=nn.Sigmoid()
        outputs=sig(outputs)
        # outputs = outputs.argmax(dim=1)
#         print(len(outputs))
#         print(len(np.max(outputs.cpu().detach().numpy(),axis=1)))
        PREDS.append(outputs.detach().cpu().numpy()) 
        # print(outputs.detach().cpu().numpy())
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = PCL_Model_Arch()
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    # print(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    # print(final_preds)
    final_preds[final_preds>=0.5] = 1
    final_preds[final_preds<0.5] = 0
    # final_preds= np.argmax(final_preds,axis=1)
    return final_preds

last 4 FC layer modified as jigsaw model

In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_all_last_8_layer_FC_asyym_loss/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_all_last_8_layer_FC_asyym_loss/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_all_last_8_layer_FC_asyym_loss/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_all_last_8_layer_FC_asyym_loss/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_all_last_8_layer_FC_asyym_loss/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds = inference(MODEL_PATH_2, valid_loader, CONFIG['device'])

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 88/88 [00:22<00:00,  3.93it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


...

100%|██████████| 88/88 [00:22<00:00,  3.99it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 88/88 [00:21<00:00,  4.01it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 88/88 [00:22<00:00,  4.00it/s]


In [ ]:
from  sklearn.metrics import f1_score,accuracy_score,precision_score,classification_report
print(classification_report(test[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].values, preds,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))

                     precision    recall  f1-score   support

            sarcasm       0.13      1.00      0.23       180
              irony       0.01      1.00      0.03        20
             satire       0.10      0.10      0.10        49
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.30      0.01        10
rhetorical_question       0.01      0.91      0.03        11

          micro avg       0.05      0.80      0.09       271
          macro avg       0.04      0.55      0.07       271
       weighted avg       0.10      0.80      0.17       271
        samples avg       0.05      0.12      0.07       271



-7:-3 with extra dropout kim cnn

In [ ]:
class Roberta_Model_Arch(nn.Module):
    def __init__(self,pre_trained='roberta-base'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained, output_hidden_states=True)
        output_channel = 16  # number of kernels
        num_classes = 6  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        ks = 3  # three conv nets here

        # input_channel = word embeddings at a value of 1; 3 for RGB images
        input_channel = 4  # for single embedding, input_channel = 1

        # [3, 4, 5] = window height
        # padding = padding to account for height of search window

        # 3 convolutional nets
        self.conv1 = nn.Conv2d(input_channel, output_channel, (3, embedding_dim), padding=(2, 0), groups=4)
        self.conv2 = nn.Conv2d(input_channel, output_channel, (4, embedding_dim), padding=(3, 0), groups=4)
        self.conv3 = nn.Conv2d(input_channel, output_channel, (5, embedding_dim), padding=(4, 0), groups=4)

        # apply dropout
        self.dropout = nn.Dropout(dropout)

        # fully connected layer for classification
        # 3x conv nets * output channel
        self.fc1 = nn.Linear(ks * output_channel, num_classes)
        self.softmax = nn.Sigmoid()

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.bert(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        hidden_layers = outputs[2]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        # 1 ,2 ,3 ,4 5, 6, 7, 8,9,10,11,12
        x = hidden_layers[:, -8:-4] 
        # x = x.unsqueeze(1)
        # x = torch.mean(x, 0)
        # print(hidden_layers.size())
      
        torch.cuda.empty_cache()
        x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        x = [F.dropout(i) for i in x]
        # max-over-time pooling; # (batch, channel_output) * ks
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # concat results; (batch, channel_output * ks)
        x = torch.cat(x, 1)
        # add dropout
        x = self.dropout(x)
        # generate logits (batch, target_size)
        logit = self.fc1(x)
        torch.cuda.empty_cache()
        return logit

In [ ]:
def inference_2(model_paths, dataloader, device,model):
    final_preds = []
    for i, path in enumerate(model_paths):
        model.to(device)
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    # print(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    # print(final_preds)
    final_preds[final_preds>=0.5] = 1
    final_preds[final_preds<0.5] = 0
    # final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds = inference_2(MODEL_PATH_2, valid_loader, CONFIG['device'],Roberta_Model_Arch())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 88/88 [00:29<00:00,  2.95it/s]


Getting predictions for model 2


100%|██████████| 88/88 [00:29<00:00,  2.98it/s]


...

100%|██████████| 88/88 [00:29<00:00,  3.00it/s]


Getting predictions for model 4


100%|██████████| 88/88 [00:29<00:00,  3.00it/s]


Getting predictions for model 5


100%|██████████| 88/88 [00:29<00:00,  3.01it/s]


In [ ]:
from  sklearn.metrics import f1_score,accuracy_score,precision_score,classification_report
print(classification_report(test[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].values, preds,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))

                     precision    recall  f1-score   support

            sarcasm       0.13      1.00      0.23       180
              irony       0.02      1.00      0.04        20
             satire       0.10      0.51      0.17        49
     understatement       0.00      0.00      0.00         1
      overstatement       0.01      0.30      0.02        10
rhetorical_question       0.05      1.00      0.09        11

          micro avg       0.07      0.88      0.13       271
          macro avg       0.05      0.64      0.09       271
       weighted avg       0.11      0.88      0.19       271
        samples avg       0.08      0.13      0.09       271



-8:-4 last layers

balaned loss + cycle sampling

In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_4_middel_layers_asymm_loss/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_4_middel_layers_asymm_loss/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_4_middel_layers_asymm_loss/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_4_middel_layers_asymm_loss/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_4_middel_layers_asymm_loss/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds = inference_2(MODEL_PATH_2, valid_loader, CONFIG['device'],Roberta_Model_Arch())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 88/88 [00:29<00:00,  2.98it/s]


Getting predictions for model 2


100%|██████████| 88/88 [00:29<00:00,  2.97it/s]


...

100%|██████████| 88/88 [00:29<00:00,  3.01it/s]


Getting predictions for model 4


100%|██████████| 88/88 [00:29<00:00,  2.99it/s]


Getting predictions for model 5


100%|██████████| 88/88 [00:29<00:00,  3.01it/s]


In [ ]:
from  sklearn.metrics import f1_score,accuracy_score,precision_score,classification_report
print(classification_report(test[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].values, preds,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))

                     precision    recall  f1-score   support

            sarcasm       0.13      1.00      0.23       180
              irony       0.02      0.95      0.04        20
             satire       0.11      0.49      0.18        49
     understatement       0.00      0.00      0.00         1
      overstatement       0.01      0.50      0.02        10
rhetorical_question       0.05      1.00      0.09        11

          micro avg       0.07      0.88      0.13       271
          macro avg       0.05      0.66      0.09       271
       weighted avg       0.11      0.88      0.19       271
        samples avg       0.07      0.13      0.09       271

